## 5.2 날씨를 이용한 매출 예측 (회귀)

### 공통 전처리

In [ ]:
# 공통 처리

# 불필요한 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

# 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 한글 글꼴 설정
import platform

if platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')
elif platform.system() == 'Darwin':
    plt.rc('font', family='Apple Gothic')

# 데이터프레임 출력용 함수
from IPython.display import display

# 숫자 출력 조정
# 넘파이 부동소수점 출력 자리수 설정
np.set_printoptions(suppress=True, precision=4)

# 판다스 부동소수점 출력 자리수 설정
pd.options.display.float_format = '{:.4f}'.format

# 데이터프레임 모든 필드 출력
pd.set_option("display.max_columns",None)

# 그래프 글꼴 크기 설정
plt.rcParams["font.size"] = 14

# 난수 시드
random_seed = 123

데이터 집합 배포 페이지 URL   
https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset

#### 데이터 필드 정보

instant 인덱스 
dteday 날짜(yy-mm-dd)  
season 계절 (1: 겨울 2: 봄 3: 여름 4: 가을)  
yr 연도 (0: 2011, 1:2012)  
mnth 월  (1 - 12)  
hr 시간  (0 - 23)  
holiday 휴일  
weekday 요일 (0 - 6)  
workingday  근무일 (1: 근무일 0: 휴일)  
weathersit 날씨 (1: 맑은뒤흐림 2: 안개 3: 약한비 4: 강한비)  
temp 기온 (정규화됨)  
atemp 체감기온 (정규화됨)  
hum 습도 (정규화됨)  
windspeed 풍속 (정규화됨)  
casual 비가입유저 이용수  
registered 가입유저 이용수  
cnt 전체유저 이용수  

### 5.2.4 데이터 읽어 들이기부터 데이터 확인까지

#### 데이터 읽어 들이기

In [ ]:
## 역자 주: wget, tail 명령에서 오류가 발생할 경우
# 데이터 집합 URL에서 파일을 직접 내려받아
# notebooks 디렉토리에 압축을 해제하고 진행하면 됩니다.

# 데이터 집합 배포 페이지 URL
url = 'https://archive.ics.uci.edu/ml/\
machine-learning-databases/00275/\
Bike-Sharing-Dataset.zip'

# 데이터를 내려받아 압축해제
!wget $url -O Bike-Sharing-Dataset.zip | tail -n 1
!unzip -o Bike-Sharing-Dataset.zip | tail -n 1

In [ ]:
# 데이터 상태 확인
!head -5 day.csv

In [ ]:
# day.csv를 데이터프레임으로 변환
# parse_dates에 날짜 칼럼을 지정한다
df = pd.read_csv('day.csv', parse_dates=[1])

# 데이터 속성 확인
print(df.dtypes)

In [ ]:
# instant 필드는 일련번호이므로 제거한다
df = df.drop('instant', axis=1)

# 우리말 필드명 정의

columns = [
    '날짜', '계절', '연도', '월', '휴일', '요일', '근무일', '날씨', 
    '온도', '체감온도', '습도', '풍속', 
    '비가입유저_이용수', '가입유저_이용수', '전체유저_이용수'
]

# 필드명을 우리말 필드명으로 변경
df.columns = columns

#### 데이터 확인

In [ ]:
# 처음 다섯 줄을 화면에 출력
display(df.head())

# 마지막 다섯 줄을 화면에 출력
display(df.tail())

In [ ]:
# 히스토그램 그리기

# 그래프 크기 조정
from pylab import rcParams
rcParams['figure.figsize'] = (12, 12)

# 데이터프레임 숫자 값 필드의 히스토그램 그리기
df.hist(bins=20)
plt.tight_layout()
plt.show()

In [ ]:
# 누락 값 확인하기
df.isnull().sum()

#### 시계열 데이터의 그래프 그리기

In [ ]:
# 시계열 데이터(가입유저 이용수) 그래프로 그리기
plt.figure(figsize=(12,4))

# 그래프 그리기
plt.plot(df['날짜'],df['가입유저_이용수'],c='b')

# 그리드 그리기
plt.grid()
plt.title('가입유저_이용수')

# 그래프 출력
plt.show()

### 5.2.5 데이터 전처리와 데이터 분할

#### 데이터 전처리
(특별한 전처리 없음)

#### 데이터 분할

방법
* '가입유저_이용수'를 목적변수 y로 삼음
* '전체유저_이용수', '비가입유저_이용수'는 대상에서 제외
* 날짜 역시 입력변수로는 불필요하므로 제외
* 시간축 범위는 2012-11-01 이전 분을 학습 데이터, 이후 분을 검증 데이터로 삼음

In [ ]:
# x와 y로 데이터를 분할
x = df.drop(['날짜', '비가입유저_이용수', '가입유저_이용수',
    '전체유저_이용수'], axis=1)
y = df['가입유저_이용수'].values

In [ ]:
# 분할 기준 날짜 mday
mday = pd.to_datetime('2012-11-1')

# 학습용 데이터의 index와 검증용 데이터의 index를 생성
train_index = df['날짜'] < mday
test_index = df['날짜'] >= mday

# 입력 데이터 분할
x_train = x[train_index]
x_test = x[test_index]

# 정답 데이터 분할
y_train = y[train_index]
y_test = y[test_index]

# 날짜 데이터 분할(그래프 출력용)
dates_test = df['날짜'][test_index]

In [ ]:
# 분할 결과 확인(모양 확인)
print(x_train.shape)
print(x_test.shape)

# 분할 결과 확인(기준값을 중심으로)
display(x_train.tail())
display(x_test.head())

In [ ]:
# 목적변수 분할결과 확인
print(y_train[:10])

### 5.2.6 알고리즘 선택하기

#### 알고리즘 선택하기

In [ ]:
# 알고리즘 선택하기 (XGBRegressor)
from xgboost import XGBRegressor
algorithm = XGBRegressor(objective ='reg:squarederror',
    random_state=random_seed)

### 5.2.7 학습 및 예측

In [ ]:
# 가입유저_이용수 예측 모델의 학습 및 예측

# 학습
algorithm.fit(x_train, y_train)

# 예측
y_pred = algorithm.predict(x_test)

# 예측 결과 확인
print(y_pred[:5])

### 5.2.8 평가

In [ ]:
# 평가(가입유저_이용수)

# score 함수를 호출
score = algorithm.score(x_test, y_test)

# R2값 계산
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, y_pred)

# 결과 확인
print(f'score: {score:.4f}  r2_score: {r2_score:.4f}')

In [ ]:
# 정답 데이터와 예측 결과의 산점도 그리기(가입유저_이용수)
plt.figure(figsize=(6,6))
y_max = y_test.max()
plt.plot((0,y_max), (0, y_max), c='k')
plt.scatter(y_test, y_pred, c='b')
plt.title(f'정답 데이터와 예측 결과의 산점도(가입유저_이용수)\
  R2={score:.4f}')
plt.grid()
plt.show()

In [ ]:
# 시계열 그래프 그리기(가입유저_이용수)
import matplotlib.dates as mdates
fig, ax = plt.subplots(figsize=(8, 4))

# 그래프 그리기
ax.plot(dates_test, y_test, label='정답 데이터', c='k')
ax.plot(dates_test, y_pred, label='예측 결과', c='b')

# 날짜 눈금 그리기
# 목요일을 기준으로 날짜 표시
weeks = mdates.WeekdayLocator(byweekday=mdates.TH)
ax.xaxis.set_major_locator(weeks)

# 날짜 표기를 세로로
ax.tick_params(axis='x', rotation=90)

# 그리드 그리기
ax.grid()
ax.legend()
ax.set_title('가입 유저 이용 수 예측')

# 그래프 출력
plt.show() 

### 5.2.9 튜닝

In [ ]:
# 필드에 원-핫 인코딩을 적용하는 함수
def enc(df, column):
    df_dummy = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df.drop([column],axis=1),df_dummy],axis=1)
    return df

# '월', '계절' 필드에 원-핫 인코딩을 적용한다

x2 = x.copy()
x2 = enc(x2, '월')
x2 = enc(x2, '계절')

# 결과 확인
display(x2.head())

In [ ]:
# 가입 유저 모델(튜닝 후)

# mday를 기준으로 입력 데이터를 분할
x2_train = x2[train_index]
x2_test = x2[test_index]

#　알고리즘 선택
algorithm2 = XGBRegressor(objective ='reg:squarederror',
    random_state=random_seed)

# 학습
algorithm2.fit(x2_train, y_train)

# 예측
y_pred2 = algorithm2.predict(x2_test)

# 예측 결과 확인
print(y_pred2[:5])

In [ ]:
# 평가(가입 유저) (튜닝 후)

# score 함수를 호출
score2 = algorithm2.score(x2_test, y_test)

# 결과 확인
print(f'score: {score2:.4f}')

In [ ]:
# 정답 데이터와 예측 결과를 산점도 비교(가입 유저)
plt.figure(figsize=(6,6))
y_max = y_test.max()
plt.plot((0,y_max), (0, y_max), c='k')
plt.scatter(y_test, y_pred2, c='b')
plt.title(f'정답 데이터와 예측 결과의 산점도(가입 유저 이용수) R2={score2:.4f}')
plt.grid()
plt.show()

In [ ]:
# 시계열 그래프 그리기(튜닝 후, 가입 유저 이용수)
import matplotlib.dates as mdates
fig, ax = plt.subplots(figsize=(8, 4))

# 그래프 그리기
ax.plot(dates_test, y_test, label='정답 데이터', c='k')
ax.plot(dates_test, y_pred, label='예측 결과 1', c='c')
ax.plot(dates_test, y_pred2, label='예측 결과 2', c='b')

# 날짜눈금 간격
weeks = mdates.WeekdayLocator(byweekday=mdates.TH)
ax.xaxis.set_major_locator(weeks)

# 날짜 형식
ax.tick_params(axis='x', rotation=90)

# 그리드 설정
ax.grid()
ax.legend()
ax.set_title('가입 유저 이용수 예측')

# 화면에 출력
plt.show()

### 5.2.10 중요도 분석

In [ ]:
# 가입유저_이용수 예측에 대한 중요도 분석
import xgboost as xgb
fig, ax = plt.subplots(figsize=(8, 4))
xgb.plot_importance(algorithm, ax=ax, height=0.8,
    importance_type='gain', show_values=False,
    title='중요도 분석(가입유저_이용수)')
plt.show()